In [12]:
#This should result in a class doing aggregate work on idle machines.
from pydantic import BaseModel
from automate_dump_load import Automated_load_dump_for_machine, Points_times
import dataloader
from schemas import Trip
from tqdm import tqdm
from datetime import datetime, timedelta
import plotly.graph_objects as go
import os, sys
import plotly.express as px
import pandas as pd
import numpy as np
import ipyleaflet as L

In [13]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [14]:
class Idle_machine(BaseModel):
    machine_id: str | int
    trips: list[Trip]
    load: Points_times = Points_times()  # Load points and times
    dump: Points_times = Points_times()  # Dump points and times
    list_of_idle_times: list[Points_times]
    total_idle_seconds: float


class Idle_machines(BaseModel):
    list_of_idle_machines: list[Idle_machine] = []


class Aggregated_idle_machines:
    
    def __init__(self, day: str, machine_type: str) -> None:
        print("Initializing...")
        #Initializing Idle_machine
        self.idle_machines = Idle_machines()
        #Need to know machines for day
        trip = dataloader.TripsLoader(day)

        print("Computing idle time for relevant machines...")
        for machine_id in tqdm(trip._machines.keys()):
            if trip._machines[machine_id].machine_type == machine_type:
                with HiddenPrints():
                    temp_automated = Automated_load_dump_for_machine(day, machine_id)
                    temp_automated.find_idle_time()
                temp_total_time_idle_seconds = sum(
                    [
                        (l.times[-1] - l.times[0]).total_seconds()
                        for l in temp_automated.stats.list_of_idle_times
                    ]
                )

                self.idle_machines.list_of_idle_machines.append(Idle_machine(
                        machine_id = temp_automated.machine.machine_id,
                        trips = temp_automated.machine.trips,
                        load = temp_automated.stats.load,
                        dump = temp_automated.stats.dump,
                        list_of_idle_times = temp_automated.stats.list_of_idle_times,
                        total_idle_seconds = temp_total_time_idle_seconds)
                    )
        print("Finished!")

    def plot_violin_idle(self):
        dict_of_idle_times_each_machine = {}
        #Plot violin idle time of each machine
        #For each machine, generate alist containing spans in seconds of each idle time
        #Plot together in a violin plot, will let us see individual differences
        for m in self.idle_machines.list_of_idle_machines:
            m.machine_id
            idle_seconds_list = [(p.times[-1]-p.times[0]).total_seconds() for p in m.list_of_idle_times]
            dict_of_idle_times_each_machine[m.machine_id] = idle_seconds_list
        
        df = pd.DataFrame.from_dict(dict_of_idle_times_each_machine, orient='index')
        df = df.transpose()

        fig = px.violin(df)

        # Customize the plot as needed (e.g., labels, title)
        fig.update_layout(
            xaxis_title="Machine ID",
            yaxis_title="Idle time in seconds",
            title="Violin Plot of idle time for machine IDs"
        )

        # Show the plot
        fig.show()

    def aggragated_nb_of_idle_timeline(self):
        
        first_timestamp = self.idle_machines.list_of_idle_machines[0].trips[0].positions[0].timestamp #First machines first timestamp
        last_timestamp = self.idle_machines.list_of_idle_machines[0].trips[-1].positions[-1].timestamp #First machines last timestamp
        for machine in self.idle_machines.list_of_idle_machines:
            if machine.trips[0].positions[0].timestamp < first_timestamp:
                first_timestamp = machine.trips[0].positions[0].timestamp
            if machine.trips[-1].positions[-1].timestamp > last_timestamp:
                last_timestamp  = machine.trips[-1].positions[-1].timestamp
        
        #Create a list of timestamps throughout day
        current_datetime = first_timestamp
        self.datetime_intervals = []

        while current_datetime < last_timestamp:
            self.datetime_intervals.append(current_datetime)
            current_datetime += timedelta(minutes=2) #This could be a parameter
        
        self.nb_of_idle_machines = [0 for i in self.datetime_intervals]
        self.nb_of_machines_in_action = [0 for i in self.datetime_intervals]
        self.nb_of_idle_waiting_for_dump = [0 for i in self.datetime_intervals]
        self.nb_of_idle_waiting_for_load = [0 for i in self.datetime_intervals]
        #Now have a list of times, and list of machines
        for i in range(len(self.datetime_intervals)):
            time = self.datetime_intervals[i]
            for m in self.idle_machines.list_of_idle_machines:
                if m.trips[0].positions[0].timestamp < time < m.trips[-1].positions[-1].timestamp:
                   self.nb_of_machines_in_action[i] += 1 
                for it in m.list_of_idle_times:
                    if it.times[0] < time < it.times[-1]:
                        self.nb_of_idle_machines[i] += 1
                        
                        #Check if we are waiting for load or dump
                        smallest_time_above = m.trips[-1].positions[-1].timestamp #Highest possible value
                        waiting_for_load = True
                        
                        for lt in m.load.times:
                            if  it.times[0] < lt < smallest_time_above:
                                smallest_time_above = lt
                        for dt in m.dump.times:
                            if it.times[0] < dt < smallest_time_above:
                                smalles_time_above = dt
                                waiting_for_load = False
                        
                        if waiting_for_load == True:
                            self.nb_of_idle_waiting_for_load[i] += 1
                        else:
                            self.nb_of_idle_waiting_for_dump[i] += 1
                        break

    def plot_aggragated_nb_of_idle_timeline(self):

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=self.datetime_intervals,
            y=self.nb_of_idle_machines,
            mode='markers+lines',
            name='Machines idle'
        ))

        fig.add_trace(go.Scatter(
            x=self.datetime_intervals,
            y=self.nb_of_machines_in_action,
            mode='markers+lines',
            name='Machines in action'
        ))

        fig.add_trace(go.Scatter(
            x=self.datetime_intervals,
            y=self.nb_of_idle_waiting_for_load,
            mode='markers+lines',
            name='Waiting for load'
        ))

        fig.add_trace(go.Scatter(
            x=self.datetime_intervals,
            y=self.nb_of_idle_waiting_for_dump,
            mode='markers+lines',
            name='Waiting for dump'
        ))


        fig.update_layout(
            title='Number of concurrently idle machines',
            xaxis_title='Time',
            yaxis_title='Machines idle',
            xaxis=dict(type='date'),
            yaxis=dict(type='linear'),
        )

        fig.show()
        fig.write_html("./data/output_html/idle_timeline.html")
    
    def plot_peak_times(self, threshold: int):

        if not len(self.datetime_intervals) > 0 :
            self.aggragated_nb_of_idle_timeline()

        
        last_val = 0 #Want to avoid plotting similar maps for same idle period
        for i in range(len(self.datetime_intervals)):
            list_of_positions = []
            list_of_load_waiting = []
            if self.nb_of_idle_machines[i] >= threshold and self.nb_of_idle_machines[i] > last_val:
                last_val = self.nb_of_idle_machines[i]
                #We are at or above threshold. Want to plot position of idle machines
                time = self.datetime_intervals[i]
                print("At: ", time)
                print("Idle machines: ", self.nb_of_idle_machines[i])
                for m in self.idle_machines.list_of_idle_machines:
                    for it in m.list_of_idle_times:
                        if it.times[0] < time < it.times[-1]:
                            list_of_positions.append(it.points[0])#Assuming its not moving a lot during this interval
                            #Check if we are waiting for load or dump
                            smallest_time_above = m.trips[-1].positions[-1].timestamp #Highest possible value
                            waiting_for_load = True
                            
                            for lt in m.load.times:
                                if  it.times[0] < lt < smallest_time_above:
                                    smallest_time_above = lt
                            for dt in m.dump.times:
                                if it.times[0] < dt < smallest_time_above:
                                    smalles_time_above = dt
                                    waiting_for_load = False
                            
                            list_of_load_waiting.append(waiting_for_load == True)
                            break
        
                # Create a map centered at the mean of all coordinates, with heatmap
                points_center = np.mean(list_of_positions, axis=0)
                m = L.Map(center=(points_center[0], points_center[1]), zoom=10)
                for k in range(len(list_of_positions)):
                    if list_of_load_waiting[k]:
                        load_icon = L.Icon(icon_url='https://cdn-icons-png.flaticon.com/512/2716/2716797.png', icon_size=[32, 32], icon_anchor=[16,16])
                        load_mark = L.Marker(location=list_of_positions[k], icon=load_icon, rotation_angle=0, rotation_origin='22px 94px')
                        m.add_layer(load_mark)
                    else:
                        dump_icon = L.Icon(icon_url='https://cdn-icons-png.flaticon.com/512/1435/1435320.png', icon_size=[32, 32], icon_anchor=[16,16])
                        dump_mark = L.Marker(location=list_of_positions[k], icon=dump_icon, rotation_angle=0, rotation_origin='22px 94px')
                        m.add_layer(dump_mark)
                # Display the map
                display(m)
                m.save('./data/output_html/my_map.html', title='PeakTime position and status')
            else:
                last_val = 0

In [15]:
day = "06-03-2022"  # MM-DD-YYYY
machine_type = 'Truck'

agg = Aggregated_idle_machines(day, machine_type)

Initializing...
Computing idle time for relevant machines...


100%|██████████| 16/16 [07:21<00:00, 27.59s/it]

Finished!


In [16]:
len(agg.idle_machines.list_of_idle_machines)

14

In [17]:
agg.aggragated_nb_of_idle_timeline()
agg.plot_aggragated_nb_of_idle_timeline()

In [18]:
agg.plot_peak_times(10)

At:  2022-06-03 13:01:36.856000+00:00
Idle machines:  10


Map(center=[59.96900319802798, 10.343488321169533], controls=(ZoomControl(options=['position', 'zoom_in_text',…

At:  2022-06-03 13:03:36.856000+00:00
Idle machines:  11


Map(center=[59.96968351353161, 10.341590873412725], controls=(ZoomControl(options=['position', 'zoom_in_text',…

At:  2022-06-03 13:05:36.856000+00:00
Idle machines:  12


...

Map(center=[59.96378558976387, 10.354812483618437], controls=(ZoomControl(options=['position', 'zoom_in_text',…

At:  2022-06-03 14:13:36.856000+00:00
Idle machines:  10


Map(center=[59.96088111433512, 10.360500062469937], controls=(ZoomControl(options=['position', 'zoom_in_text',…

At:  2022-06-03 14:41:36.856000+00:00
Idle machines:  11


Map(center=[59.96130761905527, 10.36058618633679], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [19]:
len(agg.idle_machines.list_of_idle_machines)

14

In [20]:
#Average idle seconds for each idle machine
for i in agg.idle_machines.list_of_idle_machines:
    print(i.total_idle_seconds/len(i.trips))

2542.38325
2068.150699999999
1721.028828571428
818.7772631578946
2909.7283333333335
2044.475387096774
994.1841818181812
2938.0985499999997
2657.7245909090907
1050.614476190476
3090.367052631578
892.6414285714284
3641.067071428572
901.1366874999997


In [21]:
agg.idle_machines.list_of_idle_machines[0].load

Points_times(points=[(59.95257032278229, 10.374092395817696), (59.9523698367892, 10.37432992233474), (59.95259328745276, 10.37396808828464), (59.95235110760122, 10.374403307234006), (59.949632398317085, 10.377991013879356), (59.94939676039565, 10.378258104772716), (59.94938134892647, 10.378196556144957), (59.9493903902008, 10.378204257604114), (59.94938940011622, 10.3783804987652), (59.94938679985975, 10.378219268369977), (59.94939343274386, 10.378251477405511), (59.94936780443661, 10.37826248424204), (59.94938783055366, 10.378324814321948), (59.94941015575724, 10.378277405591396), (59.94940964041925, 10.378196123140754), (59.9745062328018, 10.330139846379264), (59.949382976965005, 10.378192519885497), (59.948609866662245, 10.379689629109109), (59.94937995724837, 10.378180560775618), (59.94942766832008, 10.378419175973276), (59.94940687936055, 10.378173497150453), (59.9493818128987, 10.378168556570689), (59.94939633369317, 10.378182336196035), (59.94941636855023, 10.378109305141802)], 

In [22]:
import plotly.express as px

fig = px.scatter(x=range(10), y=range(10))
fig.show()
fig.write_html("./data/output_html/test.html")